![Credit Card](credit_card.jpg)

Commercial banks receive _a lot_ of applications for credit cards. Many of them get rejected for many reasons, like high loan balances, low income levels, or too many inquiries on an individual's credit report, for example. Manually analyzing these applications is mundane, error-prone, and time-consuming (and time is money!). Luckily, this task can be automated with the power of machine learning and pretty much every commercial bank does so nowadays. In this workbook, you will build an automatic credit card approval predictor using machine learning techniques, just like real banks do.

### The Data

The data is a small subset of the Credit Card Approval dataset from the UCI Machine Learning Repository showing the credit card applications a bank receives. This dataset has been loaded as a `pandas` DataFrame called `cc_apps`. The last column in the dataset is the target value.

### 1. Importación de Librerías

Importamos todas las bibliotecas necesarias para el análisis:
- **pandas** y **numpy**: para manipulación de datos
- **train_test_split**: para dividir los datos en conjuntos de entrenamiento y prueba
- **LabelEncoder** y **StandardScaler**: para preprocesar y escalar los datos
- **LogisticRegression**: el modelo de clasificación que utilizaremos
- **confusion_matrix** y **classification_report**: para evaluar el rendimiento del modelo
- **GridSearchCV**: para optimizar los hiperparámetros del modelo

In [33]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split as tts
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import GridSearchCV

### 2. Carga de Datos

Cargamos el dataset de aprobaciones de tarjetas de crédito desde el archivo CSV. Como el archivo no tiene encabezados, usamos `header=None`.

In [3]:
cc_apss = pd.read_csv('cc_approvals.data', header=None)

### 3. Exploración Inicial de Datos

Visualizamos las primeras filas del dataset para entender su estructura y contenido.

In [4]:
cc_apss.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,g,0,+
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,g,560,+
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,g,824,+
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,g,3,+
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,s,0,+


### 4. Información del Dataset

Obtenemos información sobre los tipos de datos, cantidad de valores no nulos y uso de memoria del dataset.

In [5]:
cc_apss.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 690 entries, 0 to 689
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       690 non-null    object 
 1   1       690 non-null    object 
 2   2       690 non-null    float64
 3   3       690 non-null    object 
 4   4       690 non-null    object 
 5   5       690 non-null    object 
 6   6       690 non-null    object 
 7   7       690 non-null    float64
 8   8       690 non-null    object 
 9   9       690 non-null    object 
 10  10      690 non-null    int64  
 11  11      690 non-null    object 
 12  12      690 non-null    int64  
 13  13      690 non-null    object 
dtypes: float64(2), int64(2), object(10)
memory usage: 75.6+ KB


### 5. Estadísticas Descriptivas - Variables Numéricas

Generamos estadísticas descriptivas (media, desviación estándar, min, max, cuartiles) para las columnas numéricas del dataset.

In [6]:
cc_apss.describe().T

,count,mean,std,min,25%,50%,75%,max
2,690.0,4.758725,4.978163,0.0,1.000,2.75,7.2075,28.0
7,690.0,2.223406,3.346513,0.0,0.165,1.00,2.6250,28.5
10,690.0,2.400000,4.862940,0.0,0.000,0.00,3.0000,67.0
12,690.0,1017.385507,5210.102598,0.0,0.000,5.00,395.5000,100000.0


### 6. Estadísticas Descriptivas - Variables Categóricas

Generamos estadísticas descriptivas (conteo, valores únicos, frecuencia) para las columnas de tipo objeto (categóricas).

In [8]:
cc_apss.describe(include=['object']).T

,count,unique,top,freq
0,690,3,b,468
1,690,350,?,12
3,690,4,u,519
4,690,4,g,519
5,690,15,c,137
6,690,10,v,399
8,690,2,t,361
9,690,2,f,395
11,690,3,g,625
13,690,2,-,383


### 7. Limpieza de Datos - Reemplazo de Valores Faltantes

Reemplazamos todos los valores '?' en el dataset con `NaN` de numpy para poder identificar y manejar los valores faltantes correctamente.

In [9]:
cc_apss.replace('?', np.nan, inplace=True)

### 8. Análisis de Valores Faltantes

Contamos cuántos valores faltantes (NaN) hay en cada columna y los ordenamos para identificar las columnas con más datos faltantes.

In [10]:
cc_apss.isna().sum().sort_values()

2      0
7      0
13     0
12     0
11     0
10     0
9      0
8      0
3      6
4      6
6      9
5      9
1     12
0     12
dtype: int64

### 9. Imputación de Valores Faltantes

Para cada columna de tipo objeto (categórica), rellenamos los valores faltantes con la moda (el valor más frecuente) de esa columna.

In [11]:
for col in cc_apss.select_dtypes(include=['object']).columns:
    moda = cc_apss[col].mode()[0]
    cc_apss[col].fillna(moda, inplace=True)

C:\Users\mauri\AppData\Local\Temp\ipykernel_34680\3837145218.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  cc_apss[col].fillna(moda, inplace=True)


### 10. Verificación de Limpieza

Verificamos que ya no queden valores faltantes en el dataset después de la imputación.

In [14]:
cc_apss.isna().sum()

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
dtype: int64

### 11. Creación de Copia del Dataset

Creamos una copia del dataset limpio para trabajar con ella sin modificar los datos originales.

In [15]:
df = cc_apss.copy()

### 12. Separación de Features y Target

Separamos el dataset en:
- **X**: las características (todas las columnas excepto la última)
- **y**: la variable objetivo (la última columna, que indica si se aprueba o no la tarjeta)

In [18]:
X = df.iloc[:, :-1]
y = df.iloc[:, -1]

### 13. Codificación de Variables

Convertimos las variables categóricas en numéricas:
- **get_dummies**: crea variables dummy para las features categóricas (one-hot encoding)
- **LabelEncoder**: convierte la variable objetivo en valores numéricos binarios (0 y 1)

In [19]:
X_encoded = pd.get_dummies(X, drop_first=True)
le = LabelEncoder()
y = le.fit_transform(y)

### 14. Conversión de Nombres de Columnas

Convertimos todos los nombres de columnas a strings para evitar problemas con el escalador y el modelo.

In [21]:
X_encoded.columns = X_encoded.columns.astype(str)

### 15. División de Datos en Entrenamiento y Prueba

Dividimos los datos en conjuntos de entrenamiento (80%) y prueba (20%). Usamos `random_state=42` para reproducibilidad.

In [22]:
X_train, X_test, y_train, y_test = tts(X_encoded, y, test_size=0.2, random_state=42)

### 16. Escalado de Features

Estandarizamos las features para que tengan media 0 y desviación estándar 1. Esto es importante para la regresión logística:
- Ajustamos el escalador con los datos de entrenamiento
- Transformamos tanto los datos de entrenamiento como los de prueba

In [24]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


### 17. Entrenamiento y Evaluación del Modelo Inicial

Creamos y entrenamos un modelo de Regresión Logística con parámetros por defecto:
- Entrenamos el modelo con los datos de entrenamiento
- Hacemos predicciones sobre los datos de prueba
- Evaluamos el rendimiento con el score de precisión, matriz de confusión y reporte de clasificación

In [29]:
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)
print(f"{logreg.score(X_test, y_test)}")
print(f"{confusion_matrix(y_test, y_pred)}")
print(f"{classification_report(y_test, y_pred)}")

0.782608695652174
[[52 18]
 [12 56]]
              precision    recall  f1-score   support

           0       0.81      0.74      0.78        70
           1       0.76      0.82      0.79        68

    accuracy                           0.78       138
   macro avg       0.78      0.78      0.78       138
weighted avg       0.79      0.78      0.78       138



### 18. Optimización de Hiperparámetros con Grid Search

Realizamos una búsqueda exhaustiva para encontrar los mejores hiperparámetros:
- **tol**: tolerancia para el criterio de parada
- **max_iter**: número máximo de iteraciones
- Usamos validación cruzada con 5 folds (cv=5)
- Optimizamos para maximizar la precisión (accuracy)
- Usamos todos los núcleos disponibles (n_jobs=-1) para acelerar el proceso

In [34]:
tol = [0.01, 0.001, 0.0001]
max_iter = [100, 200, 300, 500, 1000]
param_grid = dict(tol = tol, max_iter = max_iter)
grid_model = GridSearchCV(
    estimator = logreg,
    param_grid = param_grid,
    cv = 5,
    scoring='accuracy',
    n_jobs=-1,
)

grid_model.fit(X_train, y_train)
print(f"Mejor score: {grid_model.best_score_}")
print(f"Mejores parametros: {grid_model.best_params_}")

Mejor score: 0.846027846027846
Mejores parametros: {'max_iter': 100, 'tol': 0.001}


### 19. Evaluación del Modelo Optimizado

Obtenemos el mejor modelo encontrado por Grid Search y evaluamos su rendimiento en el conjunto de prueba para ver si los hiperparámetros optimizados mejoraron la precisión.

In [35]:
best_model = grid_model.best_estimator_
best_score = best_model.score(X_test, y_test)
print(f"Score del mejor modelo: {best_score}")

Score del mejor modelo: 0.782608695652174
